In [1]:
!pip install youtube-dl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 13.6 MB/s 


In [2]:
URL = "https://www.youtube.com/watch?v=j8AH29Ad-zU"

In [5]:
import youtube_dl

In [4]:
_OUTTMPL = '%(id)s.%(ext)s'
_YDL_OPTS = {
    "writesubtitles": True,
    "subtitlesformat": "vtt",
    "outtmpl": _OUTTMPL,
}

In [ ]:
ydl = youtube_dl.YoutubeDL(_YDL_OPTS)
ydl.download([URL])

[youtube] j8AH29Ad-zU: Downloading webpage
[youtube] j8AH29Ad-zU: Downloading MPD manifest
[info] Writing video subtitles to: Spela in och redigera i Audacity-j8AH29Ad-zU.sv.vtt
[download] Spela in och redigera i Audacity-j8AH29Ad-zU.f137.mp4 has already been downloaded
[download] 100% of 74.63MiB
[download] Resuming download at byte 9128008
[download] Destination: Spela in och redigera i Audacity-j8AH29Ad-zU.f140.m4a
[download]  93.6% of 16.48MiB at 63.08KiB/s ETA 00:16